In [21]:
import requests
import sys
import time
import numpy as np
from fastai.vision import parallel

Big Buck Bunny: 12 min 1080p 30fps.

In [2]:
test_url = "http://distribution.bbb3d.renderfarming.net/video/mp4/bbb_sunflower_1080p_30fps_normal.mp4"

A smaller video I found for testing.

In [3]:
test_url_2 = "https://cdn.videvo.net/videvo_files/video/premium/video0036/small_watermarked/cog_clock17_preview.mp4"

In [4]:
time_1 = time.time()

In [5]:
time.time() - time_1

0.012939929962158203

It's in seconds--Linux epoch divided by 1000.

In [6]:
# !wget 'http://distribution.bbb3d.renderfarming.net/video/mp4/bbb_sunflower_1080p_30fps_normal.mp4' -O bbb.mp4

Nice one-liner from https://stackoverflow.com/questions/2017843/fetch-frame-count-with-ffmpeg.

In [7]:
# !ffprobe -v error -select_streams v:0 -show_entries stream=nb_frames -of default=nokey=1:noprint_wrappers=1 -ignore_editlist 1 bbb.mp4

In [38]:
frame_count = 19036

In [39]:
fps = 30

In [41]:
minutes = frame_count / fps / 60
minutes

10.575555555555555

In [9]:
mb_count = 276134947 / (1e+6)

In [10]:
frame_size = mb_count / frame_count
frame_size

0.01450593333683547

`frame_size` represents the average MB size of frame.

In [11]:
second_size = frame_size * fps
second_size

0.4351780001050641

`second_size` represents the average MB size of a second at 30 FPS.

In [13]:
(round(0.2555 * 10000)/100)

25.55

In [14]:
using_url = test_url

In [ ]:
process_count = 4

In [15]:
from multiprocessing import Process, Queue

mbps_queue = Queue()
mbps_percent_queue = Queue()

def download_measure(i):
    global mbps_list
    response = requests.get(using_url, stream=True)
    total_length = response.headers.get('content-length')

    start = time.time()
    dl = 0
    total_length = int(total_length)
    last_print_time = 0
    for data in response.iter_content(chunk_size=1024):
        dl += len(data)
        done = int(50 * dl / total_length)
        dl_mb = dl/(1e+6)
        if time.time() - last_print_time > 0.2:
            last_print_time = time.time()
            mbps_percent_queue.put((i, f"{i}: {round((dl/total_length) * 10000)/100}%"))
        
    elapsed = time.time() - start
    mbps_queue.put((total_length/1e+6)/elapsed)

In [16]:
processes = [Process(target=download_measure, args=(i,)) for i in range(process_count)]

In [17]:
from queue import Empty

def print_progress(): 
    try:
        progress_outputs = sorted([mbps_percent_queue.get(block=False) for _ in processes], key=lambda a: a[0] if a else 0)
        out_str = ', '.join([p[1] for p in filter(lambda p: p, progress_outputs)])
        sys.stdout.write(f"\r{' ' * len(out_str)}")
        sys.stdout.write(f"\r{out_str}")
        sys.stdout.flush()  
    except Empty:
        pass

In [19]:
# Run processes
for p in processes:
    p.start()

#Exit the completed processes
while True in [p.is_alive() for p in processes]:
    print_progress()
    
# for p in processes:
#     p.join()

0: 99.88%, 0: 99.91%, 0: 99.93%, 0: 99.96%

KeyboardInterrupt: 

In [20]:
mbps_averages = [mbps_queue.get() for p in processes]
mbps_averages

[0.11428740806678182,
 0.10982495573134107,
 0.10970769706001597,
 0.10905637498324859]

In [24]:
mbps_average = np.average(mbps_averages)
mbps_average

0.11071910896034687

In [45]:
relative_length_ratio = second_size / mbps_average
relative_length_ratio

3.9304687708507435

In [46]:
extra_length_proportion = relative_length_ratio - 1
extra_length_proportion

2.9304687708507435

In [50]:
minutes_spent_buffering = extra_length_proportion * minutes
minutes_spent_buffering

30.99133528995264

That's an absurdly high number.